In [6]:
player_name = input("What is the name of the player you want to search for? ")
platform = input("What platform is the player on? ")
type = input("What is the type of the player? ")

In [7]:
processed_player_name = player_name.replace(" ", "+")
processed_platform = platform.lower()
processed_type = type.lower()

In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os

query = f"https://www.bcci.tv/search?platform={processed_platform}&type={processed_type}&term={processed_player_name}&content_type=all"

if not processed_player_name or not processed_platform or not processed_type:
    print("Invalid input")
    exit()
else:
    try:
        response = requests.get(query)
        response.raise_for_status()
    except requests.RequestException as e:
        print(e)

    soup = BeautifulSoup(response.text, 'html.parser')
    data = []

    player_data = soup.find('div', class_='lv pc video-section-append-video-here')
    if player_data:
        player_card = player_data.find_all(
            'div', class_='slick-card m-0 lv-bg hoverVideoPlayNow')
        for card in player_card:
            header_content = card.find('div', class_='bottom')
            title_holder = None
            title = ""
            date = ""
            views = ""
            if header_content.find('div', class_='text-detail br-b'):
                title_holder = header_content.find('div', class_='text-detail br-b')
                date_holder = header_content.find('div', class_='text-detail br-b')
                date =  date_holder.find('span').text
                views_holder = header_content.find(
                    'div', class_='tour-overlay-details')
                list = views_holder.find('ul')
                views = list.find_all('li')[0]
                views = views.find('span', class_="me-3").text
                views = views.replace("&nbsp;", "").replace(
                    "views", "").replace("\n", "").replace("\xa0", "").strip()
                if "k" in views:
                    views = views.replace("k", "")
                    views = float(views) * 1000
                elif "m" in views:
                    views = views.replace("m", "")
                    views = float(views) * 1000000
            elif header_content.find('div', class_='text-detail pb-0'):
                title_holder = header_content.find('div', class_='text-detail pb-0')
                date_holder = header_content.find(
                    'div', class_='tour-overlay-details')
                ul = date_holder.find('ul')
                date_ = ul.find_all('li')[0]
                date = date_.find('span').text
                views = "-"
            if title_holder:
                title = title_holder.find('p').text

            data.append({
                "title": title,
                "date": date,
                "views": views,
                "platform": processed_platform,
                "type": processed_type,
                "player_name": player_name,
                "sport": "Cricket"
            })
        if data:
            df = pd.DataFrame(data)
            header = ['title', 'date', 'views', 'platform', 'type', 'player_name']
            if os.path.isfile('Sports.csv') and os.path.getsize('Sports.csv') > 0:
                df.to_csv('Sports.csv', mode='a', header=False, index=False)
            else:
                df.to_csv('Sports.csv', mode='a', header=header, index=False)
    else:
        print("Player not found")

Invalid input


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

class ICC_Scrapper:
    def __init__(self):
        self.url = "https://www.icc-cricket.com/search?"
        self.player_name = None

    def get_player_data(self, player_name):
        processed_player_name = player_name.replace(" ", "%20")

        if not processed_player_name:
            return {"Error": "Invalid input"}

        self.player_name = processed_player_name

        url = self.url + "q=" + self.player_name

        response = self.get_data(processed_player_name, url)

        if response:
            return {"Response": response}
        else:
            return {"Error": "No data found"}

    def get_data(self, player_name, url):
        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.RequestException as e:
            return {"Error": str(e)}

        soup = BeautifulSoup(response.text, 'html.parser')
        data = []

        player_data = soup.find(
            'div', class_='my-4 grid grid-cols-1 lg:grid-cols-4 gap-6 lg:gap-x-6 lg:gap-y-10')
        if player_data:
            cards = player_data.find_all(
                'div', class_='h-[541px] relative rounded-lg lg:rounded-[14px] overflow-hidden')
            for card in cards:
                title = ""
                date = ""
                content = ""
                image_url = ""
                link = ""
                link_holder = card.find('a')
                link = link_holder['href']
                image_holder = link_holder.find_all(
                    'div')[0]
                image_holder = image_holder.find('picture')
                image_holder = image_holder.find('img')
                image_url = image_holder['src']
                divs = link_holder.find_all('div')
                if len(divs) > 1:
                    content_holder = divs[1]
                    content_holder = content_holder.find('div')
                    title_holder = content_holder.find_all('div')[1] if len(
                        content_holder.find_all('div')) > 1 else None
                    if title_holder:
                        title = title_holder.text
                    content_holder = content_holder.find(
                        'div', class_='text-sm font-bold text-white leading-[1.2] lg:text-lg lg:leading-[1.4] lg:-tracking-[0.72px]')
                    if content_holder:
                        content = content_holder.text
                    date_holder = content_holder.find('time')
                    date = date_holder.text if date_holder else ""
                else:
                    continue

                temp_player_name = player_name.replace("%20", " ")
                data.append({
                    "title": title,
                    "date": date,
                    "content": content,
                    "player_name": temp_player_name,
                    "image_url": image_url,
                    "link": link,
                    "sport": "Cricket"
                })

            if data:
                df = pd.DataFrame(data)
                header = ['title', 'date', 'content',
                          'player_name', 'image_url', 'link', 'sport']
                if os.path.isfile('ICC.csv') and os.path.getsize('ICC.csv') > 0:
                    df.to_csv('ICC.csv', mode='a', header=False, index=False)
                else:
                    df.to_csv('ICC.csv', mode='a', header=header, index=False)
                return data
            else:
                return {"Error": "No data found"}
        else:
            return {"Error": "No data found"}

In [2]:
scrapper = ICC_Scrapper()

# player_name = "MS Dhoni"
# scrapper.get_player_data(player_name)

player_name = "Sachin Tendulkar"
scrapper.get_player_data(player_name)

# player_name = "Sourav Ganguly"
# scrapper.get_player_data(player_name)

# player_name = "Ravindra Jadeja"
# scrapper.get_player_data(player_name)

# player_name = "Jasprit Bumrah"
# scrapper.get_player_data(player_name)

# player_name = "Mithali Raj"
# scrapper.get_player_data(player_name)

# player_name = "Smriti Mandhana"
# scrapper.get_player_data(player_name)

# player_name = "Jhulan Goswami"
# scrapper.get_player_data(player_name)

# player_name = "Harmanpreet Kaur"
# scrapper.get_player_data(player_name)

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

class FIFA_Scrapper:
    def __init__(self):
        self.url = "https://www.fifa.com/en/search?"
        self.player_name = None

    def get_player_data(self, player_name):
        processed_player_name = player_name.replace(" ", "%20")

        if not processed_player_name:
            return {"Error": "Invalid input"}

        self.player_name = processed_player_name

        url = self.url + "q=" + self.player_name

        response = self.get_data(processed_player_name, url)

        if response:
            return {"Response": response}
        else:
            return {"Error": "No data found"}

    def get_data(self, player_name, url):
        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.RequestException as e:
            return {"Error": str(e)}

        soup = BeautifulSoup(response.text, 'html.parser')
        data = []

        player_data = soup.find(
            'div', class_='search-results-page_searchResults__pKRI-')
        if player_data:
            title = ""
            date = ""
            content = ""
            image_url = ""
            link = ""
            player_name = ""
            sport = "Football"
            cards = player_data.find_all('a')
            for card in cards:
                link = card['href']
                date_holder = card.find(
                    'div', class_='search-result-card_details__+1JIM')
                date = date_holder.find('span')[1].text
                image_holder = card.find(
                    'div', class_='search-result-card_imageContainer__NgxbS')
                image_holder = image_holder.find(
                    'div', class_='image_imgContainer__nDjya')
                image_holder = image_holder.find('img')
                image_url = image_holder['src']
                content_holder = card.find(
                    'div', class_='search-result-card_textContainer__82BrL')
                title_holder = content_holder.find(
                    'div', class_='search-result-card_title__CTmi4')
                title = title_holder.text
                content_holder = content_holder.find(
                    'div', class_='search-result-card_description__KiKIZ search-result-card_smallCardDescription__InIop')
                content = content_holder.text
                temp_player_name = player_name.replace("%20", " ")
                data.append({
                    "title": title,
                    "date": date,
                    "content": content,
                    "player_name": temp_player_name,
                    "image_url": image_url,
                    "link": link,
                    "sport": sport
                })

            if data:
                df = pd.DataFrame(data)
                header = ['title', 'date', 'content',
                          'player_name', 'image_url', 'link', 'sport']
                if os.path.isfile('FIFA.csv') and os.path.getsize('FIFA.csv') > 0:
                    df.to_csv('FIFA.csv', mode='a', header=False, index=False)
                else:
                    df.to_csv('FIFA.csv', mode='a', header=header, index=False)
                return data
            else:
                return {"Error": "No data found"}
        else:
            return {"Error": "No data found"}

In [4]:
scrapper = FIFA_Scrapper()

player_name = "Lionel Messi"
scrapper.get_player_data(player_name)

{'Response': {'Error': '401 Client Error: Unauthorized for url: https://www.fifa.com/en/search?q=Lionel%20Messi'}}